In [98]:
import numpy as np
import pandas as pd
from pathlib import Path
pd.options.display.max_columns = None

train_df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv', low_memory=False)
test_df = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
train_df.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.00000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.00000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.00000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.00000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.00000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.00000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.51452,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.66469,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.00000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.00000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [99]:
# First let's store our dependent variable 'target'
depVar = train_df['target']
# Remove non-common columns
common = list(set(train_df.columns).intersection(set(test_df.columns)))
train_df = train_df[common]
test_df = test_df[common]
train_df.head()

,tbp_lv_nevi_confidence,tbp_lv_H,tbp_lv_deltaLB,tbp_lv_A,tbp_lv_Hext,tbp_lv_x,tbp_lv_area_perim_ratio,tbp_lv_stdL,tbp_lv_deltaL,tbp_lv_stdLExt,tbp_lv_norm_color,tbp_lv_norm_border,tbp_lv_B,tbp_lv_eccentricity,tbp_lv_location_simple,tbp_lv_deltaA,tbp_lv_areaMM2,copyright_license,tbp_lv_deltaLBnorm,age_approx,isic_id,tbp_tile_type,tbp_lv_Aext,sex,tbp_lv_deltaB,tbp_lv_perimeterMM,tbp_lv_location,tbp_lv_radial_color_std_max,tbp_lv_symm_2axis,attribution,tbp_lv_z,tbp_lv_C,tbp_lv_Cext,tbp_lv_color_std_mean,tbp_lv_y,anatom_site_general,clin_size_long_diam_mm,tbp_lv_Bext,tbp_lv_L,tbp_lv_minorAxisMM,tbp_lv_symm_2axis_angle,patient_id,tbp_lv_Lext,image_type
0,2.628592e-03,53.058545,8.360566,20.244422,55.828924,-182.703552,27.476170,2.036195,-7.658253,2.637780,0.000000,7.091360,26.922447,0.901302,Right Leg,3.982447,3.152561,CC-BY,5.784302,60.0,ISIC_0015670,3D: white,16.261975,male,2.967674,9.307003,Right Leg - Upper,0.00000,0.590476,Memorial Sloan Kettering Cancer Center,-42.427948,33.684638,28.953117,0.00000,613.493652,lower extremity,3.04,23.954773,54.367448,1.543016,85,IP_1235828,62.025701,TBP tile: close-up
1,1.334303e-07,39.702910,6.839008,31.712570,44.064040,-0.078308,12.235290,0.853227,-6.500838,3.912844,0.000000,2.116402,26.331000,0.639885,Head & Neck,6.347830,0.919497,CC-BY,4.987244,60.0,ISIC_0015845,3D: white,25.364740,male,1.781713,3.354148,Head & Neck,0.00000,0.285714,Memorial Sloan Kettering Cancer Center,57.174500,41.219030,35.299260,0.00000,1575.687000,head/neck,1.10,24.549290,48.861520,0.821918,55,IP_8170065,55.362360,TBP tile: close-up
2,2.959177e-04,59.265850,9.092376,22.575830,62.909730,123.649700,24.184620,1.743651,-7.709336,1.950777,0.000000,4.798335,37.970460,0.932147,Torso Back,5.447655,3.265153,CC-BY,6.290359,60.0,ISIC_0015864,3D: XP,17.128170,male,4.485044,8.886309,Torso Back Top Third,0.00000,0.361905,Memorial Sloan Kettering Cancer Center,232.908900,44.174920,37.611800,0.00000,1472.010000,posterior torso,3.40,33.485410,53.961180,1.194905,105,IP_6724798,61.670520,TBP tile: close-up
3,2.198945e+01,56.414429,4.783413,14.242329,60.060388,-141.024780,14.889242,1.258541,-4.665323,1.573733,1.771705,1.975874,21.448144,0.654458,Torso Front,2.077572,6.079940,CC-0,6.400196,65.0,ISIC_0015902,3D: XP,12.164757,male,0.326788,9.514499,Torso Front Top Half,0.66469,0.209581,ACEMID MIA,58.359802,25.746200,24.374023,0.51452,1442.185791,anterior torso,3.22,21.121356,18.649518,2.481328,130,IP_4111386,23.314841,TBP tile: close-up
4,1.378832e-03,46.946070,9.148495,24.725520,52.041180,-72.315640,19.902560,2.085409,-8.579431,2.480509,0.000000,3.658854,26.464900,0.946448,Torso Front,4.668053,2.101708,CC-BY,6.531302,55.0,ISIC_0024200,3D: white,20.057470,male,0.754434,6.467562,Torso Front Top Half,0.00000,0.313433,Memorial Sloan Kettering Cancer Center,21.428960,36.217980,32.608740,0.00000,1488.720000,anterior torso,2.73,25.710460,46.276310,0.929916,20,IP_8313778,54.855740,TBP tile: close-up


In [100]:
train_df.dtypes

tbp_lv_nevi_confidence         float64
tbp_lv_H                       float64
tbp_lv_deltaLB                 float64
tbp_lv_A                       float64
tbp_lv_Hext                    float64
tbp_lv_x                       float64
tbp_lv_area_perim_ratio        float64
tbp_lv_stdL                    float64
tbp_lv_deltaL                  float64
tbp_lv_stdLExt                 float64
tbp_lv_norm_color              float64
tbp_lv_norm_border             float64
tbp_lv_B                       float64
tbp_lv_eccentricity            float64
tbp_lv_location_simple          object
tbp_lv_deltaA                  float64
tbp_lv_areaMM2                 float64
copyright_license               object
tbp_lv_deltaLBnorm             float64
age_approx                     float64
isic_id                         object
tbp_tile_type                   object
tbp_lv_Aext                    float64
sex                             object
tbp_lv_deltaB                  float64
tbp_lv_perimeterMM       

In [101]:
# Drop useless columns
train_df.drop(columns=['copyright_license', 'isic_id'], inplace=True)

In [102]:
for col in train_df.columns:
    if (train_df[col].isnull().sum() > 0):
        print(col, " ", train_df[col].isnull().sum())

age_approx   2798
sex   11517
anatom_site_general   5756


In [103]:
# Fill 'age_approx' missing with mean of the column
train_df['age_approx'] = train_df['age_approx'].fillna(train_df['age_approx'].median())
# Fill 'sex' and 'anatom_site_general'
sex_mode = train_df['sex'].mode()[0]
train_df['sex'] = train_df['sex'].fillna(sex_mode)
anatom_mode = train_df['anatom_site_general'].mode()[0]
train_df['anatom_site_general'] = train_df['anatom_site_general'].fillna(anatom_mode)

In [104]:
print("There are", train_df.isnull().sum().sum(), "missing value in train_data")
print("There are", test_df.isnull().sum().sum(), "missing value in test_data")

There are 0 missing value in train_data
There are 0 missing value in test_data


In [105]:
numerical_columns = []
categorical_columns = []
for col in train_df.columns:
    if train_df[col].dtype != 'object':
        numerical_columns.append(col)
    else:
        categorical_columns.append(col)

In [106]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])
test_df[numerical_columns] = scaler.fit_transform(test_df[numerical_columns])

In [107]:
categorical_columns

['tbp_lv_location_simple',
 'tbp_tile_type',
 'sex',
 'tbp_lv_location',
 'attribution',
 'anatom_site_general',
 'patient_id',
 'image_type']

In [108]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

for col in categorical_columns:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = LE.fit_transform(train_df[col])
    test_df[col] = test_df[col].astype(str)
    test_df[col] = LE.fit_transform(test_df[col])

In [109]:
from sklearn.model_selection import train_test_split
columns = categorical_columns + numerical_columns
X = train_df[columns]
y = depVar
X.head()

,tbp_lv_location_simple,tbp_tile_type,sex,tbp_lv_location,attribution,anatom_site_general,patient_id,image_type,tbp_lv_nevi_confidence,tbp_lv_H,tbp_lv_deltaLB,tbp_lv_A,tbp_lv_Hext,tbp_lv_x,tbp_lv_area_perim_ratio,tbp_lv_stdL,tbp_lv_deltaL,tbp_lv_stdLExt,tbp_lv_norm_color,tbp_lv_norm_border,tbp_lv_B,tbp_lv_eccentricity,tbp_lv_deltaA,tbp_lv_areaMM2,tbp_lv_deltaLBnorm,age_approx,tbp_lv_Aext,tbp_lv_deltaB,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_symm_2axis,tbp_lv_z,tbp_lv_C,tbp_lv_Cext,tbp_lv_color_std_mean,tbp_lv_y,clin_size_long_diam_mm,tbp_lv_Bext,tbp_lv_L,tbp_lv_minorAxisMM,tbp_lv_symm_2axis_angle,tbp_lv_Lext
0,4,1,1,12,4,2,120,0,-0.928564,-0.288931,-0.315920,0.067613,-0.917620,-0.910543,1.566233,-0.390639,0.361297,0.639823,-1.51192,2.110595,-0.257500,1.112661,-0.405730,-0.556591,-0.729588,0.145624,0.380443,0.721606,-0.434492,-1.383632,2.268546,-1.116897,-0.192995,-0.407543,-1.403047,-1.039738,-0.511069,-0.659882,1.111639,-0.849629,-0.025344,0.894687
1,0,1,1,0,4,1,843,0,-0.928627,-2.708061,-0.754827,2.935019,-3.006591,0.015277,-1.278352,-1.071225,0.695485,2.683578,-1.51192,-0.774183,-0.369545,-0.704539,0.489257,-0.787296,-1.061012,0.145624,2.959583,0.186393,-1.440162,-1.383632,-0.168817,0.015359,1.126869,0.906538,-1.403047,1.308111,-1.624050,-0.527265,0.604861,-1.464287,-0.596126,0.344021
2,5,0,1,16,4,3,676,0,-0.928620,0.835409,-0.104823,0.650540,0.339647,0.642518,0.951893,-0.558946,0.346547,-0.461349,-1.51192,0.780962,1.835454,1.327077,0.148659,-0.544959,-0.519164,0.145624,0.625867,1.406380,-0.505564,-1.383632,0.440524,2.013064,1.644677,1.385391,-1.403047,1.055128,-0.304536,1.466075,1.074246,-1.146356,0.355177,0.865335
3,6,0,1,19,0,0,397,0,-0.398516,0.318926,-1.347782,-1.433104,-0.166282,-0.699252,-0.783014,-0.838039,1.225467,-1.065700,-0.64548,-0.855669,-1.294561,-0.603234,-1.126475,-0.254154,-0.473493,0.514637,-0.780446,-0.470201,-0.399438,-0.478837,-0.777702,0.028833,-1.583639,-1.355725,-0.728635,0.982354,-0.407803,-1.291919,-2.175924,-0.049818,0.830829,-2.304422
4,6,1,1,19,4,0,863,0,-0.928594,-1.396095,-0.088635,1.188031,-1.590171,-0.350931,0.152681,-0.362325,0.095318,0.387739,-1.51192,0.120223,-0.344179,1.426487,-0.146318,-0.665158,-0.418978,-0.223390,1.455843,-0.277209,-0.914185,-1.383632,0.052864,-0.390987,0.250792,0.349419,-1.403047,1.095902,-0.688917,-0.268249,0.366912,-1.372231,-1.262039,0.302153


In [110]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [111]:
# Hyper Parameter Optimization
params = {
    "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],  # Step size shrinkage to prevent overfitting
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],  # Maximum depth of a tree
    "min_child_weight": [1, 3, 5, 7],  # Minimum sum of instance weight (hessian) needed in a child
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],  # Minimum loss reduction required to make a further partition
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7, 0.8],  # Subsample ratio of columns when constructing each tree
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of the training instances
    "n_estimators": [100, 200, 300, 400, 500],  # Number of boosting rounds
    "reg_alpha": [0, 0.01, 0.05, 0.1, 1, 10],  # L1 regularization term on weights
    "reg_lambda": [0.01, 0.05, 0.1, 1, 10],  # L2 regularization term on weights
    "scale_pos_weight": [1, 1.5, 2, 2.5, 3],  # Controls the balance of positive and negative weights
    "colsample_bylevel": [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns for each split
}

In [112]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

regressor = xgboost.XGBRegressor()
random_search = RandomizedSearchCV(regressor, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)
random_search.fit(X, y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bylevel=0.7, colsample_bytree=0.4, gamma=0.5, learning_rate=0.25, max_depth=3, min_child_weight=3, n_estimators=200, reg_alpha=10, reg_lambda=10, scale_pos_weight=1, subsample=0.7;, score=0.611 total time=  18.9s
[CV 5/5] END colsample_bylevel=0.7, colsample_bytree=0.4, gamma=0.5, learning_rate=0.25, max_depth=3, min_child_weight=3, n_estimators=200, reg_alpha=10, reg_lambda=10, scale_pos_weight=1, subsample=0.7;, score=0.734 total time=  18.6s
[CV 4/5] END colsample_bylevel=0.8, colsample_bytree=0.4, gamma=0.5, learning_rate=0.15, max_depth=15, min_child_weight=3, n_estimators=400, reg_alpha=0.01, reg_lambda=0.05, scale_pos_weight=2, subsample=0.9;, score=0.915 total time=  32.6s
[CV 5/5] END colsample_bylevel=0.9, colsample_bytree=0.4, gamma=0.2, learning_rate=0.2, max_depth=10, min_child_weight=7, n_estimators=100, reg_alpha=0.01, reg_lambda=0.1, scale_pos_weight=3, subsample=0.8;, score=0.786 total t

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 3/5] END colsample_bylevel=0.7, colsample_bytree=0.4, gamma=0.5, learning_rate=0.25, max_depth=3, min_child_weight=3, n_estimators=200, reg_alpha=10, reg_lambda=10, scale_pos_weight=1, subsample=0.7;, score=0.704 total time=  19.0s
[CV 3/5] END colsample_bylevel=0.8, colsample_bytree=0.4, gamma=0.5, learning_rate=0.15, max_depth=15, min_child_weight=3, n_estimators=400, reg_alpha=0.01, reg_lambda=0.05, scale_pos_weight=2, subsample=0.9;, score=0.859 total time=  31.5s
[CV 5/5] END colsample_bylevel=0.8, colsample_bytree=0.4, gamma=0.5, learning_rate=0.15, max_depth=15, min_child_weight=3, n_estimators=400, reg_alpha=0.01, reg_lambda=0.05, scale_pos_weight=2, subsample=0.9;, score=0.888 total time=  32.0s
[CV 4/5] END colsample_bylevel=0.7, colsample_bytree=0.4, gamma=0.5, learning_rate=0.25, max_depth=3, min_child_weight=3, n_estimators=200, reg_alpha=10, reg_lambda=10, scale_pos_weight=1, subsample=0.7;, score=0.707 total time=  18.9s
[CV 2/5] END colsample_bylevel=0.8, colsample_

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.15,
                                                          0.2, 0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'reg_alpha': [0, 0.01, 0.05, 0.1, 1,
                                                      10],
                                        'reg_lambda': [0.01, 0.05, 0.1, 1, 10],
                                        'scale_pos_weight': [1, 1.5, 2, 2.5, 3],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   scoring='roc_auc', verbose=3)

In [113]:
random_search.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.8, colsample_bynode=None, colsample_bytree=0.4,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.5, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.15, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [114]:
import xgboost
from xgboost import XGBRegressor

model = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=1.0, colsample_bynode=None, colsample_bytree=0.4,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.3, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.15, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=3, missing=np.nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=None)
model.fit(X_train,y_train)

In [124]:
valid_predictions = model.predict(X_valid)
valid_predictions

array([0.00061075, 0.00061075, 0.00061075, ..., 0.00061075, 0.00061075,
       0.00061075], dtype=float32)

In [125]:
submission_csv = pd.DataFrame({
    'id': X_valid['patient_id'],
    'class': valid_predictions
})

solution_csv = pd.DataFrame({
    'id': X_valid['patient_id'],
    'target': y_valid
})

In [127]:
"""
2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""

import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

#     # Equivalent code that uses sklearn's roc_auc_score
#     v_gt = abs(np.asarray(solution.values)-1)
#     v_pred = np.array([1.0 - x for x in submission.values])
#     max_fpr = abs(1-min_tpr)
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
#     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
#     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

score(solution_csv, submission_csv, 'id')

0.07616805128799875

In [119]:
# Reordering the columns to be able to predict
test_df.drop(columns=['isic_id', 'copyright_license'], inplace=True)
cols_when_model_builds = model.get_booster().feature_names
test_df = test_df[cols_when_model_builds]

In [120]:
test_preds = model.predict(test_df)

In [121]:
test_df = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
submission_csv = pd.DataFrame({
    'isic_id': test_df['isic_id'],
    'target': test_preds
})
submission_csv.head()

,isic_id,target
0,ISIC_0015657,0.000611
1,ISIC_0015729,0.010861
2,ISIC_0015740,0.000611


In [122]:
submission_csv.to_csv('submission.csv', index=False)